# Dispatch design pattern

**Used for regulating *interactions* between objects.**

Create a mini game with three types of Heroes: Dwarf, Elf and Human. Now create a class Fight that allows them to interact with the class Monster.

The Heroes hit differently the monsters, according to this table.

| | Troll | Giant |
|---|---|---|
| Dwarf | 6 | 6 |
| Elf | 3 | 1 |
| Human | 3 | 3 |

Now add the class Bat (Monster) that is hit by them in this way:

| | Bat | 
|---|---|
| Dwarf | 0 |
| Elf | 0 |
| Human | 3 |

With the double dispatch method it is easy to add a new monster without changing the Hero interface.

Now let's make it work with the following main code.

In [5]:
from abc import ABC, abstractmethod

class Hero(ABC):
    @abstractmethod
    def fight_against(self, monster):
        pass 

class Dwarf(Hero):
    def fight_against(self, monster):
        monster._fight_against_dwarf()

class Elf(Hero):
    def fight_against(self, monster):
        monster._fight_against_elf()

class Human(Hero):
    def fight_against(self, monster):
        monster._fight_against_human()


In [10]:
class Monster(ABC):
    @abstractmethod
    def _fight_against_dwarf(self):
        pass
    
    @abstractmethod
    def _fight_against_elf(self):
        pass

    @abstractmethod
    def _fight_against_human(self):
        pass


class Troll(Monster):
    def _fight_against_dwarf(self):
        print("Dwarf hits Troll with 6 HP.")
    
    def _fight_against_elf(self):
        print("Elf hits Troll with 3 HP.")

    def _fight_against_human(self):
        print("Human hits Troll with 3 HP.")


class Giant(Monster):
    def _fight_against_dwarf(self):
        print("Dwarf hits Giant with 6 HP.")
    
    def _fight_against_elf(self):
        print("Elf hits Giant with 1 HP.")

    def _fight_against_human(self):
        print("Human hits Giant with 3 HP.")


class Bat(Monster):
    def _fight_against_dwarf(self):
        print("Dwarf hits Bat with 0 HP.")
    
    def _fight_against_elf(self):
        print("Elf hits Bat with 0 HP.")

    def _fight_against_human(self):
        print("Human hits Bat with 3 HP.")


In [11]:
d = Dwarf()
e = Elf()
h = Human()

t = Troll()
g = Giant()
b = Bat()

d.fight_against(b)
e.fight_against(g)
h.fight_against(t)

Dwarf hits Bat with 0 HP.
Elf hits Giant with 1 HP.
Human hits Troll with 3 HP.


# Strategy design pattern

**Used to make a class behave differently depending on the situation.**

Create a class Queue that handles a queue of strings that are passed to it and prints them according to different strategies. Use an abstract class ProcessingStrategy that can be used as a base for the following strategies:

* FIFO: the first element that was added is the first to be printed.
* LIFO: the last element that was added is the first to be printed.
* RANDOM: prints an element without caring who arrived first.

Now create a new strategy that handles the queue by printing the strings in alphabetical order.

Now let's make it work with the following main code.

Extra exercise: you can implement it also using functions rather than classes.


In [13]:
class Queue:
    def __init__(self, strategy):
        self.strategy = strategy
        self.queue = []

    def add_item(self, message):
        self.queue.append(message)
    
    def process(self):
        if len(self.queue) > 0:
            # returns the index of the next element to process
            next_item_index = self.strategy._process(self.queue)
            
            # item = self.queue[next_item_index]
            # del self.queue[next_item_index]
            item = self.queue.pop(next_item_index)
            print(item)
        else:
            print("no items in the queue!")


class Strategy(ABC):
    @abstractmethod
    def _process(self, queue):
        """Returns the index of the next element to process."""
        pass

class FIFOStrategy(Strategy):
    def _process(self, queue):
        return 0
        
class LIFOStrategy(Strategy):
    def _process(self, queue):
        return -1

import random
class RandomStrategy(Strategy):
    def _process(self, queue):
        indexes = list(range(len(queue)))

        # gives me a random index
        return random.choice(indexes)


In [16]:
q = Queue(strategy=RandomStrategy())

q.add_item("first item")
q.add_item("second item")
q.add_item("third item")

for _ in range(3):
    q.process()

third item
first item
second item




# State design pattern

**Decides behavior of an object depending of its internal state.**

 Implement a class State that manages the HTTP response code depending on whether a webpage is loaded.

The function has to return the status code depending on the state of the page and the received request.

The pages can be created (CREATE), requested (GET), updated (UPDATE), or deleted (DELETE).

Draw the state machine and create a class that implements the state machine.

It should work with the following main.


In [20]:
class ProcessRequest:
    def __init__(self, page_name):
        self.page_name = page_name
        self.state = StateExisting()
    
    def process_input(self, request):
        self.state._process_input(self, request)
    
    def _set_state(self, new_state):
        self.state = new_state

class HTTPState(ABC):
    def _process_input(self, page, request):
        self._action(page, request)
        self._change_state(page, request)

    @abstractmethod
    def _action(self, page, request):
        pass

    @abstractmethod
    def _change_state(self, page, request):
        pass

class StateExisting(HTTPState):
    def _action(self, page, request):
        responses = {
            "GET": [page.page_name, "get", 200],
            "UPDATE": [page.page_name, "update", 200],
            "DELETE": [page.page_name, "delete", 200],
            "CREATE": [page.page_name, "create", 400],
        }
        if request not in responses:
            print("Request not allowed.")
        print(responses[request])

    def _change_state(self, page, request):
        if request == "DELETE":
            page._set_state(StateNotExisting())

class StateNotExisting(HTTPState):
    def _action(self, page, request):
        responses = {
            "GET": [page.page_name, "get", 404],
            "UPDATE": [page.page_name, "update", 404],
            "DELETE": [page.page_name, "delete", 400],
            "CREATE": [page.page_name, "create", 202],
        }
        if request not in responses:
            print("Request not allowed.")
        else:
            print(responses[request])

    def _change_state(self, page, request):
        if request == "CREATE":
            page._set_state(StateExisting())



In [21]:
server = ProcessRequest('home')  # creates a page named "home"
server.process_input('GET')  # get the page, 200
server.process_input('UPDATE')  # update the page, 200
server.process_input('DELETE')  # delete the page, 200 --> transition to StateNotExisting
server.process_input('GET')  # not existing, 404
server.process_input("POST")  # request not allowed
server.process_input('CREATE')  # create the page, 202 --> transition to StateExisting
server.process_input('GET')  # get the page, 200

['home', 'get', 200]
['home', 'update', 200]
['home', 'delete', 200]
['home', 'get', 404]
Request not allowed.
['home', 'create', 202]
['home', 'get', 200]


# Observer design pattern

**Used for updating multiple subscribers of any changes that happen in the publisher.**

Write a class Welcome that updates the mailing list, a telegram chat, and a slack chat of the creation of new users in a company.

It has to work with the following main.


In [ ]:
company = CompanyUpdate(events=['new entry'])

telegram = TelegramSubscriber()
email = MailSubscriber()
database = DataBaseSubscriber()

# subscribe all to the event "new entry"
company.register('new entry', telegram, telegram.send_message)
company.register('new entry', email, email.send_mail)
company.register('new entry', database)

# send a message
company.dispatch('new entry', 'Bob')

# the company stops using telegram and starts using slack
slack = SlackSubscriber()
company.unregister('new entry', telegram)
company.register('new entry', slack, slack.send_message)

company.dispatch('new entry', 'Alice')

# Singleton design pattern

**Used for creating a single instance of an object across multiple parts of the code.**

Create a singleton that handles a logger (with an attribute `name``) and returns it every time the class is used. Use a metaclass for implementing it.
The method `log` should print the name of the logger (stored during the initialization of the class) and the message passed to the logger.

It should work with the following main.



In [ ]:
logger1 = Logger("logger 1")
logger2 = Logger("logger 2")

logger2.log("message to logger 2")

# Factory design pattern

**Used for separating creation from use.**

Define a class Country that creates a class for speaking the language and a class for paying a specified amount as a currency of the country. 
Create the countries Italy, England, and USA.

The script should work with the following main.

In [ ]:
for factory in [Italy(), England(), USA()]:
    # retrieve the language and currency
    language = factory.get_language()
    currency = factory.get_currency()

    language.speak()
    currency.pay(20)
